# Keras - Save Model Architecture and Weights

This is a reference notebook to demonstrate the ability to save models built using Keras.

For simplicity I have previously pickled the MNIST dataset into the required structure for inputting directly to a Keras MLP model. I will build a simple 2 layered MLP with a softmax output layer. Purpose of this model is solely for testing save functionality.

In [29]:
import pickle
import os
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

path = '/Users/Nick/Documents/Enrion/Datasets/'

X_train = pickle.load( open(os.path.join(path,"mnist_X_train.pkl"), "rb" ) )
X_test = pickle.load( open(os.path.join(path,"mnist_X_test.pkl"), "rb" ) )
y_train = pickle.load( open(os.path.join(path,"mnist_y_train.pkl"), "rb" ) )
y_test = pickle.load( open(os.path.join(path,"mnist_y_test.pkl"), "rb" ) )

In [ ]:
np.random.seed(0)

model = Sequential()

model.add(Dense(input_dim=X_train.shape[1], 
                output_dim=50, 
                init='normal', 
                activation='tanh'))

model.add(Dense(input_dim=50, 
                output_dim=50, 
                init='normal', 
                activation='tanh'))

model.add(Dense(input_dim=50, 
                output_dim=y_train.shape[1], 
                init='normal', 
                activation='softmax'))

sgd = SGD(lr=0.001, decay=1e-7, momentum=.9, nesterov=False)

model.compile(optimizer = sgd, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=50, batch_size=300, validation_split=0.1, verbose=1, show_accuracy=True)

In [39]:
predictions = model.predict_classes(X_test, batch_size = 300, verbose =0)
test_acc = np.sum(predictions == y_test) / len(y_test)
print("Test set accuracy score: {}%".format(test_acc*100))

Test set accuracy score: 94.1%


### Save Architecture of Model and Weights

The skeleton of the model is saved to a JSON file using the model.to_json() function

The weights must be saved in a HDF5 file using the h5py library. HDF5 is a data library that stores files as binary data. The h5py package is a Pythonic interface to the HDF5 binary data format.

In [33]:
model_skeleton = model.to_json()
open('my_model_architecture.json', 'w').write(model_skeleton)
model.save_weights('my_model_weights.h5')

### Load Model and Weights

Import the model_from_json function from keras.models and declare a name for the loaded model
Load the weights into the imported model from the .h5 file

In [41]:
from keras.models import model_from_json
loaded_model = model_from_json(open('my_model_architecture.json').read())
loaded_model.load_weights('my_model_weights.h5')

The new model must first be compiled prior to use

In [42]:
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9, nesterov=False)

loaded_model.compile(optimizer = sgd, loss='categorical_crossentropy', metrics=['accuracy'])

### Test the loaded model

The loaded model should yield the same accuracy as the saved model. We can see below that it does indeed yield 94.1% accuracy on the MNIST test set.

In [43]:
predictions_2 = loaded_model.predict_classes(X_test, batch_size = 300, verbose =0)
test_acc_2 = np.sum(predictions_2 == y_test) / len(y_test)
print("Test set accuracy score from loaded model: {}%".format(test_acc_2*100))

Test set accuracy score from loaded model: 94.1%
